In [1]:
# DATA format
'''

# Retrieve Data
# Data format. Each item in the list represents userid, video_id, num of times viewed / rating
user_ad_views = [[1,1,5],
    [3,2,1],
                 [3,1,1],
                 [1,2,10],
                 [2,4,1],
       [2,1,6],
       [2,2,2],
       [3,3,1]]
user_ad_views = sc.parallelize(user_ad_views)
user_ad_views = user_ad_views.map(lambda x:(x[0], x[1], x[2]))

ad_names = [(1,"one"),(2,"two"),(3,"three"),(4,"four")]
ad_names = sc.parallelize(ad_names)
# print (user_ad_views.collect())

'''

[(1, 1, 5), (3, 2, 1), (3, 1, 1), (1, 2, 10), (2, 4, 1), (2, 1, 6), (2, 2, 2), (3, 3, 1)]


In [4]:
# MongoDB connection
import pymongo

client = pymongo.MongoClient('mongodb://152.46.20.123:27017')
db = client.testgridfs

# USERS data
users = db.users
data = []
for obj in users.find():
    if(len(obj['videosViewed']) >= 1):
        for viewed in obj['videosViewed']:
            data.append((obj['userId'], viewed['videoId'], 1))
print (data)

# VIDEOS information
videos = db.videos
ad_names = []
for obj in videos.find():
    ad_names.append((obj['videoId'], obj['title']))
print (ad_names)


[(1, 1, 1), (2, 1, 1), (2, 6, 1), (2, 7, 1), (2, 2, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1), (2, 6, 1), (2, 7, 1), (2, 8, 1), (2, 9, 1), (2, 10, 1), (3, 1, 1), (3, 7, 1), (4, 1, 1), (4, 7, 1), (4, 4, 1), (4, 5, 1), (4, 10, 1), (5, 6, 1), (5, 7, 1), (5, 10, 1), (6, 1, 1), (6, 7, 1), (6, 2, 1), (6, 4, 1), (6, 8, 1), (7, 1, 1), (7, 6, 1), (7, 5, 1), (7, 6, 1), (7, 7, 1), (7, 8, 1), (7, 10, 1), (8, 1, 1), (8, 10, 1), (9, 5, 1), (9, 6, 1), (9, 7, 1), (9, 8, 1), (10, 1, 1), (10, 6, 1), (10, 7, 1), (10, 9, 1)]
[(1, u'laptop1'), (2, u'laptop2'), (3, u'clothing1'), (4, u'clothing2'), (5, u'med1'), (6, u'grocery1'), (7, u'sport1'), (8, u'sport2'), (9, u'footwear1'), (10, u'footwear2'), (11, u'laptop4'), (12, u'laptop3'), (13, u'cothing3'), (14, u'clothing4'), (15, u'cos1'), (16, u'cos3'), (17, u'movie1'), (18, u'movie2'), (19, u'rest1'), (20, u'rest2'), (21, u'groc1'), (22, u'groc2'), (23, u'sport3'), (24, u'mobile5')]


In [5]:
# Forming user-ads-views RDDs
user_ad_views = sc.parallelize(data)
user_ad_views = user_ad_views.map(lambda x:((x[0], x[1]), x[2]))
user_ad_views = user_ad_views.reduceByKey(lambda x, y: x+y)
user_ad_views = user_ad_views.map(lambda x:(x[0][0], x[0][1], x[1]))
print (user_ad_views.take(5))

# Forming ad-names RDDS
ad_names = sc.parallelize(ad_names)
print (ad_names.take(2))

[(6, 4, 1), (6, 8, 1), (4, 10, 1), (2, 8, 1), (7, 7, 1)]
[(1, u'laptop1'), (2, u'laptop2')]


In [6]:
# Data split for cross validation
trainData, validationData, testData = user_ad_views.randomSplit([40, 40, 20], 13)
print (trainData.collect())

[(6, 8, 1), (4, 10, 1), (8, 10, 1), (9, 5, 1), (3, 7, 1), (6, 7, 1), (7, 8, 1), (2, 3, 1), (9, 6, 1), (4, 1, 1), (2, 6, 2), (3, 1, 1), (5, 7, 1), (10, 6, 1), (4, 4, 1), (6, 2, 1), (4, 7, 1), (2, 9, 1), (6, 1, 1), (2, 1, 1), (10, 1, 1), (10, 9, 1)]


In [7]:
unique_user = user_ad_views.map(lambda x:(x[0],(x[1], x[2]))).groupByKey().map(lambda x:x[0])
unique_ad = ad_names.map(lambda x:x[0])
all_pairs = unique_user.cartesian(unique_ad)
trainData_m = trainData.map(lambda x:(x[0], x[1]))
print (unique_user.take(5))
print (unique_ad.take(5))
print (all_pairs.take(5))
print (trainData_m.take(5))

[8, 4, 1, 5, 9]
[1, 2, 3, 4, 5]
[(8, 1), (8, 2), (8, 3), (8, 4), (8, 5)]
[(6, 8), (4, 10), (8, 10), (9, 5), (3, 7)]


In [8]:
# Evaluating a model

def computeAvg(rdd):
    userid, pre, to_c = rdd[0], list(rdd[1][0]), list(rdd[1][1])
    to_c = sc.parallelize(to_c).map(lambda x:x[0]).collect()
    X = len(to_c)
    pre = sc.parallelize(pre).sortBy(lambda x:-x[1]).map(lambda x:x[0]).collect()[:X]
    same = 0
    for i in pre:
        if i in to_c: same+=1
    return same*1.0/X
        
    
def modelEval(model, validationData):
    to_be_predicted = all_pairs.subtract(trainData_m)
    predicted_data = model.predictAll(to_be_predicted).map(lambda x:(x[0], (x[1], x[2]))).groupByKey()
    to_compare = validationData.subtract(trainData).map(lambda x:(x[0], (x[1], x[2]))).groupByKey()
    merge = predicted_data.join(to_compare)
    total = 0
    for each in merge.collect():
        total+=computeAvg(each)
    return (total*1.0/50)

In [9]:
# Building the model
from pyspark.mllib.recommendation import *
import random
from operator import *

for rank in [1, 2, 3]:
    model = ALS.trainImplicit(trainData, rank = rank, seed = 345)
    modelScore = modelEval(model, validationData)
    print "The model score for rank %d is %f"%(rank, modelScore)

The model score for rank 1 is 0.046667
The model score for rank 2 is 0.006667
The model score for rank 3 is 0.060000


In [10]:
bestModel = ALS.trainImplicit(trainData, rank=1, seed=345)
ans = modelEval(bestModel, testData)
print "Model Score for best model, rank = 10 is %f"%(ans)


Model Score for best model, rank = 10 is 0.046667


In [37]:
# Recommend for a single user
top = bestModel.recommendProducts(2, 2)
print (top)

[Rating(user=2, product=1, rating=1.1812443516124844), Rating(user=2, product=6, rating=0.6369860890532593)]


In [11]:
# Recommendations for all users - concurrently and process each one
def ratingstoList(ratings):
    l = []
    for each in ratings:
        l.append(each[1])
    return l
recommendations_raw = bestModel.recommendProductsForUsers(3)
print (recommendations_raw.take(2))
recommendations = recommendations_raw.map(lambda x: (x[0], ratingstoList(x[1])))
print (recommendations.take(2))

[(2, (Rating(user=2, product=1, rating=0.9209411627055033), Rating(user=2, product=7, rating=0.643245298477666), Rating(user=2, product=6, rating=0.39742814094009304))), (3, (Rating(user=3, product=1, rating=0.794531378603125), Rating(user=3, product=7, rating=0.5549524708809983), Rating(user=3, product=6, rating=0.34287655010354356)))]
[(2, [1, 7, 6]), (3, [1, 7, 6])]


In [59]:
# get ad name using the video Id
'''
client = pymongo.MongoClient('mongodb://152.46.20.123:27017')
db = client.testgridfs
ads = db.videos
temp = ads.find_one({"videoId": 1})
print (temp['title'])
'''

laptop1


In [13]:
# Store back the recommendations into Database
def store_function(x):
    client = pymongo.MongoClient('mongodb://152.46.20.123:27017')
    db = client.testgridfs
    users = db.users
    ads = db.videos
    rec = []
    for recommendation in x[1]:
        temp = ads.find_one({"videoId": recommendation})
        rec.append(temp)
    users.find_one_and_update({"userId": x[0]}, 
                                 {"$set": {"recommendations":rec}})
    return 1
    
store = recommendations.map(lambda x: store_function(x))
print (store.collect())

[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [3]:
client = pymongo.MongoClient('mongodb://152.46.20.123:27017')
db = client.testgridfs
users = db.users
ads = db.videos
rec = []
temp = ads.find_one({"videoId": 3})
rec.append((3, temp['title'], temp))
users.find_one_and_update({"userId": 2}, 
                                 {"$set": {"recommendations":rec}})

{u'__v': 13,
 u'_id': ObjectId('5ade031c5c7eae0014d001cf'),
 u'coins': 0,
 u'emailId': u'user1@gmail.com',
 u'password': u'testtest',
 u'recommendations': [[1, u'laptop1'], [6, u'grocery1'], [4, u'clothing2']],
 u'time': 655963.72,
 u'userId': 2,
 u'username': u'user1',
 u'videosLiked': [{u'_id': ObjectId('5ade032d5c7eae0014d001d0'),
   u'videoId': 1}],
 u'videosViewed': [{u'_id': ObjectId('5ade060f5c7eae0014d001d4'),
   u'videoId': 1},
  {u'_id': ObjectId('5ade066b5c7eae0014d001d5'), u'videoId': 6},
  {u'_id': ObjectId('5ade06f75c7eae0014d001d6'), u'videoId': 7},
  {u'_id': ObjectId('5ade07415c7eae0014d001d7'), u'videoId': 2},
  {u'_id': ObjectId('5ade07815c7eae0014d001d8'), u'videoId': 3},
  {u'_id': ObjectId('5ade07c05c7eae0014d001d9'), u'videoId': 4},
  {u'_id': ObjectId('5ade080c5c7eae0014d001da'), u'videoId': 5},
  {u'_id': ObjectId('5ade082e5c7eae0014d001db'), u'videoId': 6},
  {u'_id': ObjectId('5ade089c5c7eae0014d001dc'), u'videoId': 7},
  {u'_id': ObjectId('5ade08df5c7eae0014